In [1]:
import re

#### 1-Eliminate implication

In [106]:
#done
def E_I(exp):

    pat = r'(\b\w+\b)\s*->\s*(\b\w+\b)'
    return re.sub(pat, r' ~\1 | \2 ', exp)

logical_expression = "(P -> Q)"
eliminated_expression = E_I(logical_expression)
print("Original expression:", logical_expression)
print("Expression with implications eliminated:", eliminated_expression)


Original expression: (P -> Q)
Expression with implications eliminated: ( ~P | Q )


#### 2-Move negation inward (Demorgan Law)

In [97]:
def MNI(expression):
    # Loop until all negations (~) have been moved inward
    while True:
        # Find the start index of the next negation (~)
        negation_start = expression.find('~(')
        # If no negation is found, exit the loop
        if negation_start == -1:
            break
        
        # Find the corresponding closing parenthesis for the negation
        counter = 1
        negation_end = negation_start + 2
        for i in range(negation_start + 2, len(expression)):
            if expression[i] == '(':
                open_paren_count += 1
            elif expression[i] == ')':
                open_paren_count -= 1
                if open_paren_count == 0:
                    negation_end = i
                    break
        
        # Extract the inner formula enclosed by the negation
        inner_formula = expression[negation_start + 2:negation_end]
        
        # Split the inner formula by the connective symbols (| or &)
        subExp = inner_formula.split('|') if '|' in inner_formula else inner_formula.split('&')
        
        # Apply negation to each subformula and reconstruct the inner formula
        negated_subformulas = ['~' + subformula if subformula[0] != '~' else subformula[1:] for subformula in subExp]
        inner_formula = '(' + '|'.join(negated_subformulas) + ')' if '| ' in inner_formula else '(' + '& '.join(negated_subformulas) + ')'
        
        # Replace the negated expression with the transformed inner formula
        expression = expression[:negation_start] + inner_formula + expression[negation_end + 1:]
    
    # Return the expression with all negations moved inward
    return expression

# Example usage:
formula = "∀x(~(P(x) & Q(x)) | ~R(x))"
moved_formula = MNI(formula)
print("Original formula:", formula)
print("Formula with negation moved inward:", moved_formula)


Original formula: ∀x(~(P(x) & Q(x)) | ~R(x))
Formula with negation moved inward: ∀x((~P(x) & ~ Q(x)) | ~R(x))


####  3-Remove double-not.

In [67]:
#done
def remove_doube(exp):
    result = []
    i = 0
    while i < len(exp):
        if exp[i:i+2] == '~~':
            # Skip over the double negation
            i += 2
        else:
            result.append(exp[i])
            i += 1
    return ''.join(result)

logical_expression = "~~(P & Q) "
expression_without_double_negation = remove_doube(logical_expression)
print("Original expression:", logical_expression)
print("Expression without double negation:", expression_without_double_negation)

Original expression: ~~(P & Q) 
Expression without double negation: (P & Q) 


####  4-Standardize variable scope

In [78]:
def standardize_variable_scope(formula):
  """Renames variables within quantifiers to ensure unique names."""

  def rename_variables_in_scope(scope):
    """Renames variables within a quantifier scope."""
    counter = 0
    new_scope = ""
    for i, char in enumerate(scope):
      if char.isalpha():
        if i > 0 and scope[i-1] == "∀" or scope[i-1] == "∃":
          counter += 1  # Reset counter for new quantifier
        new_scope += char + f"{counter}"  # Append unique identifier
      else:
        new_scope += char
    return new_scope

  i = 0
  while i < len(formula):
    if formula[i] in "∀∃":
      quantifier_end = formula.find(")", i)
      scope = formula[i+1:quantifier_end].strip()
      new_scope = rename_variables_in_scope(scope)
      formula = formula[:i+1] + new_scope + formula[quantifier_end:]
      i += len(new_scope) + 2  # Move past quantifier and scope
    else:
      i += 1

  return formula

expressions = ['∃x(P(x) ∧ Q(x))', '∀y(R(y) ∨ S(y))']
print("Original Expressions:", expressions)
print("SVS Result:", standardize_variable_scope(expressions))


Original Expressions: ['∃x(P(x) ∧ Q(x))', '∀y(R(y) ∨ S(y))']
SVS Result: ['∃x(P(x) ∧ Q(x))', '∀y(R(y) ∨ S(y))']


In [75]:
def SVS(exp):
    l = []
    for i in exp:
        copy_exp = i
        rep_map = {}
        
        def new_var(old_var, used_vars):
            if old_var in rep_map:
                return rep_map[old_var]
            new_vars = set(used_vars)
            varNew = chr(ord(max(new_vars)) + 1) if new_vars else 'a'
            while varNew in new_vars:
                varNew = chr(ord(varNew) + 1)
            new_vars.add(varNew)
            rep_map[old_var] = varNew
            return varNew

        i = 0
        while i < len(copy_exp):
            if copy_exp[i] in "∀∃":
                variable = copy_exp[i + 1]
                if variable.islower(): # Confirm variable
                    new_vari = new_var(variable, copy_exp[:i])  # Use current expression's variables
                    copy_exp = copy_exp[:i + 1] + copy_exp[i + 1:].replace(variable, new_vari)
                    i += len(new_vari) - 1
            i += 1
        l.append(copy_exp)
    return l

# Test case
expressions = ['∃x(P(x) ∧ Q(x))', '∀y(R(y) ∨ S(y))']
print("Original Expressions:", expressions)
print("SVS Result:", SVS(expressions))


Original Expressions: ['∃x(P(x) ∧ Q(x))', '∀y(R(y) ∨ S(y))']
SVS Result: ['∃b(P(b) ∧ Q(b))', '∀b(R(b) ∨ S(b))']


In [50]:
def SVS(exp):
    l = []
    for i in exp:
        copy_exp = i
        var = set()
        rep_map = {}
    def new_var(old_var):
        if old_var in rep_map:
            return rep_map[old_var]
        varNew = chr(ord('a') + len(var))
        while  varNew in var:
            varNew = chr(ord(var)+1)
        var.add(varNew)
        rep_map[old_var] = varNew
        return varNew
    i = 0
    while i < len(copy_exp):
        if copy_exp[i] in "∀∃":
            variable = copy_exp[i+1]
            if variable.islower(): # confirm variable
                new_vari = new_var(variable)
                copy_exp = copy_exp[:i+1] + copy_exp[i+1:].replace(variable,new_vari)
                i += len(new_vari) - 1
        i+=1
    l.append(copy_exp)
    

#### 5- The prenex form (obtained by moving all quantifiers to the left of the
formula.)

In [66]:
#done
def penex_form(exp):
    quanti_part = ""
    parts = ""
    i = 0
    while i < len(exp):
        if exp[i] == '∀' or exp[i] == '∃':
            quanti_part += exp[i] + exp[i+1] + ' '
            i += 1  # Skip the next character which is the variable
        else:
            parts += exp[i]
        i += 1
    return quanti_part.strip() + " " + parts.strip()

# Test case
expression = '∃x(P(x) ∧ Q(x)) ∧ ∀y(R(y) ∨ S(y))'
print("Original Expression:", expression)
print("Penex Form:", penex_form(expression))


Original Expression: ∃x(P(x) ∧ Q(x)) ∧ ∀y(R(y) ∨ S(y))
Penex Form: ∃x ∀y (P(x) ∧ Q(x)) ∧ (R(y) ∨ S(y))


#### 6-Skolemization for existential quantifiers

In [ ]:
def skolemization(exp):
    parts = exp.split('∀')
    skolemized_parts = [parts[0]]
    if part in parts[1:]

In [60]:
def skolemization(exp):
    parts = exp.split('∃')
    skolemized_parts = [parts[0]]
    for part in parts[1:]:
        if part.strip():
            variables, inner_exp = part.split(')')
            variables += ')'  # Adding back the closing parenthesis
            skolem_function = "f" + variables.replace('(', '').replace(')', '')
            skolemized_inner_exp = inner_exp.replace('(', '(' + skolem_function + '(')
            skolemized_parts.append(skolemized_inner_exp)
    return ''.join(skolemized_parts)

# Example usage:
logical_expression = "∃x (P(x) & Q(x))"
skolemized_expression = skolemization(logical_expression)
print("Original expression:", logical_expression)
print("Skolemized expression:", skolemized_expression)


ValueError: too many values to unpack (expected 2)

#### 7-Eliminate universal quantifiers.

In [71]:
# done
def E_U(expression):
    # Define a pattern to match universal quantifiers and their corresponding variables and inner expressions
    pattern = r'∀(\w+)\s*\((.*?)\)'

    # Use regex to remove all occurrences of universal quantifiers
    expression_without_universal_quantifiers = re.sub(pattern, r'(\2)', expression)

    return expression_without_universal_quantifiers

# Example usage:
logical_expression = "∀x (P(x) & Q(x))"
expression_without_universal_quantifiers = E_U(logical_expression)
print("Original expression:", logical_expression)
print("Expression without universal quantifiers:", expression_without_universal_quantifiers)


Original expression: ∀x (P(x) & Q(x))
Expression without universal quantifiers: (P(x) & Q(x))
